In [ ]:
import glob
import os

In [ ]:
# owner = "chrisfilo"
# dataset = "urbansound8k"

owner = "sylkaladin"
dataset = "speech-commands-v2"

flattened_files = False

In [ ]:
datapath = f"../data/{dataset}"
raw_datapath = f"{datapath}_raw"

In [ ]:
import glob
import os

# Download the UrbanSound8K dataset
if not os.path.exists(datapath):
    os.makedirs(datapath)
    !kaggle datasets download {owner}/{dataset}
    !mkdir -p {datapath}
    !mkdir -p {raw_datapath}
    !mv {dataset}.zip {raw_datapath}
    !unzip {raw_datapath}/{dataset} -d {raw_datapath}
    !rm {raw_datapath}/{dataset}.zip

    cnt = 0
    sum = len(glob.glob(f"{raw_datapath}/*/*"))
    for file in glob.glob(f"{raw_datapath}/*/*"):
        fixed_file = datapath + "/" + file.split("/")[-1] if flattened_files else file.replace(raw_datapath, datapath)
        os.makedirs(os.path.dirname(fixed_file), exist_ok=True)
        os.system(f"ffmpeg -i {file} -acodec pcm_s16le -ar 16000 {fixed_file} > /dev/null 2>&1")
        cnt += 1
        if cnt % 100 == 0:
            print(f"Processed {cnt/sum} files")
    !rm -rf {raw_datapath}